In [1]:
import os
import math
import mxnet as mx
from mxnet import image
from mxnet import nd, gluon, autograd, init
from mxnet.gluon.data.vision import ImageFolderDataset
from mxnet.gluon.data import DataLoader
from mxnet.gluon import nn
from tensorboardX import SummaryWriter
import numpy as np
import shutil

In [2]:
def transform_train(data, label):
    im = data.asnumpy()
    im = np.pad(im, ((4, 4), (4, 4), (0, 0)), mode='constant', constant_values=0)
    im = nd.array(im, dtype='float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), resize=0, rand_mirror=True,
                                    rand_crop=True,
                                   mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1)) # channel x width x height
    return im, nd.array([label]).astype('float32')

def transform_test(data, label):
    im = data.astype('float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1))
    return im, nd.array([label]).astype('float32')

In [3]:
train_ds = ImageFolderDataset('./data/train_data/', transform=transform_train)
valid_ds = ImageFolderDataset('./data/valid_data/', transform=transform_test)
train_valid_ds = ImageFolderDataset('./data/train_valid/', transform=transform_train)
test_ds = ImageFolderDataset('./data/TestSet/', transform=transform_test)

In [18]:
train_data = DataLoader(train_ds, batch_size=64, shuffle=True, last_batch='keep')
valid_data = DataLoader(valid_ds, batch_size=64, shuffle=True, last_batch='keep')
train_valid_data = DataLoader(train_valid_ds, batch_size=128, shuffle=True, last_batch='keep')
test_data = DataLoader(test_ds, batch_size=128, shuffle=False, last_batch='keep')

In [5]:
criterion = gluon.loss.SoftmaxCrossEntropyLoss()

In [6]:
from resnet import ResNet164_v2

In [9]:
model = ResNet164_v2(10)
model.initialize(ctx=mx.gpu(1), init=mx.initializer.Xavier())
model.hybridize()

In [21]:
import datetime
writer = SummaryWriter()

def get_acc(output, label):
    pred = output.argmax(1, keepdims=True)
    correct = (pred == label).sum()
    return correct.asscalar()

def train(net, train_data, valid_data, num_epochs, lr, wd, ctx, lr_decay):
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd', {'learning_rate': lr, 'momentum': 0.9, 'wd': wd})

    prev_time = datetime.datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0
        correct = 0
        total = 0
        if epoch == 89 or epoch == 139:
            trainer.set_learning_rate(trainer.learning_rate * lr_decay)
        for data, label in train_data:
            bs = data.shape[0]
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            with autograd.record():
                output = net(data)
                loss = criterion(output, label)
            loss.backward()
            trainer.step(bs)
            train_loss += nd.mean(loss).asscalar()
            correct += get_acc(output, label)
            total += bs
        writer.add_scalars('loss', {'train': train_loss / len(train_data)}, epoch)
        writer.add_scalars('acc', {'train': correct / total}, epoch)
        cur_time = datetime.datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_correct = 0
            valid_total = 0
            valid_loss = 0
            for data, label in valid_data:
                bs = data.shape[0]
                data = data.as_in_context(ctx)
                label = label.as_in_context(ctx)
                output = net(data)
                loss = criterion(output, label)
                valid_loss += nd.mean(loss).asscalar()
                valid_correct += get_acc(output, label)
                valid_total += bs
            valid_acc = valid_correct / valid_total
            writer.add_scalars('loss', {'valid': valid_loss / len(valid_data)}, epoch)
            writer.add_scalars('acc', {'valid': valid_acc}, epoch)
            epoch_str = ("Epoch %d. Train Loss: %f, Train acc %f, Valid Loss: %f, Valid acc %f, "
                         % (epoch, train_loss / len(train_data),
                            correct / total, valid_loss / len(valid_data), valid_acc))
        else:
            epoch_str = ("Epoch %d. Loss: %f, Train acc %f, "
                         % (epoch, train_loss / len(train_data),
                            correct / total))
        prev_time = cur_time
        print(epoch_str + time_str + ', lr ' + str(trainer.learning_rate))

In [23]:
ctx = mx.gpu(1)
num_epochs = 200
learning_rate = 0.1
weight_decay = 1e-4
lr_decay = 0.1
train(model, train_valid_data, None, num_epochs, learning_rate, weight_decay, ctx, lr_decay)

Epoch 0. Loss: 1.475502, Train acc 0.456540, Time 00:03:25, lr 0.1
Epoch 1. Loss: 0.962582, Train acc 0.654680, Time 00:03:18, lr 0.1
Epoch 2. Loss: 0.768715, Train acc 0.730140, Time 00:03:20, lr 0.1
Epoch 3. Loss: 0.646319, Train acc 0.774780, Time 00:03:20, lr 0.1
Epoch 4. Loss: 0.556304, Train acc 0.806260, Time 00:03:20, lr 0.1
Epoch 5. Loss: 0.508641, Train acc 0.824540, Time 00:03:16, lr 0.1
Epoch 6. Loss: 0.472446, Train acc 0.837760, Time 00:03:21, lr 0.1
Epoch 7. Loss: 0.431642, Train acc 0.851300, Time 00:03:16, lr 0.1
Epoch 8. Loss: 0.401767, Train acc 0.861060, Time 00:03:21, lr 0.1
Epoch 9. Loss: 0.378359, Train acc 0.869960, Time 00:03:17, lr 0.1
Epoch 10. Loss: 0.360752, Train acc 0.874180, Time 00:03:22, lr 0.1
Epoch 11. Loss: 0.341468, Train acc 0.881440, Time 00:03:19, lr 0.1
Epoch 12. Loss: 0.326856, Train acc 0.886580, Time 00:03:18, lr 0.1
Epoch 13. Loss: 0.311444, Train acc 0.891360, Time 00:03:15, lr 0.1
Epoch 14. Loss: 0.303571, Train acc 0.894080, Time 00:03:1

Epoch 115. Loss: 0.005066, Train acc 0.998940, Time 00:03:14, lr 0.010000000000000002
Epoch 116. Loss: 0.004368, Train acc 0.999260, Time 00:03:16, lr 0.010000000000000002
Epoch 117. Loss: 0.004636, Train acc 0.999060, Time 00:03:14, lr 0.010000000000000002
Epoch 118. Loss: 0.003994, Train acc 0.999300, Time 00:03:13, lr 0.010000000000000002
Epoch 119. Loss: 0.003640, Train acc 0.999460, Time 00:03:14, lr 0.010000000000000002
Epoch 120. Loss: 0.003481, Train acc 0.999480, Time 00:03:13, lr 0.010000000000000002
Epoch 121. Loss: 0.003952, Train acc 0.999280, Time 00:03:14, lr 0.010000000000000002
Epoch 122. Loss: 0.003550, Train acc 0.999500, Time 00:03:14, lr 0.010000000000000002
Epoch 123. Loss: 0.003462, Train acc 0.999480, Time 00:03:15, lr 0.010000000000000002
Epoch 124. Loss: 0.003724, Train acc 0.999400, Time 00:03:15, lr 0.010000000000000002
Epoch 125. Loss: 0.003183, Train acc 0.999640, Time 00:03:16, lr 0.010000000000000002
Epoch 126. Loss: 0.003549, Train acc 0.999320, Time 00

In [24]:
model.save_params('./resnet.params')

In [25]:
import pandas as pd

preds = []
for data, _ in test_data:
    data = data.as_in_context(ctx)
    output = model(data)
    pred_label = output.argmax(1)
    preds.extend(pred_label.astype(int).asnumpy())

sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key = lambda x:str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_ds.synsets[x])
df.to_csv('submission.csv', index=False)